In [18]:
import os, sys, glob, argparse
import pandas as pd
import numpy as np
from tqdm import tqdm

import cv2, time
# from PIL import Image
# from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchmetrics
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset


# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [19]:
# 读取数据集
train_path = glob.glob('./data/train/*')
test_path = glob.glob('./data/test/*')

train_path.sort()
test_path.sort()

train_df = pd.read_csv('data/train.csv')
train_df = train_df.sort_values(by='name')
train_label = train_df['label'].values

# 自定义数据集
# 带有图片缓存的逻辑
DATA_CACHE = {}
class XunFeiDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label = img_label
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
    def __getitem__(self, index):
        if self.img_path[index] in DATA_CACHE:
            img = DATA_CACHE[self.img_path[index]]
        else:
            img = cv2.imread(self.img_path[index])
            DATA_CACHE[self.img_path[index]] = img
        if self.transform is not None:
            img = self.transform(image = img)['image']
        img = img.transpose([2,0,1])
        return img, torch.from_numpy(np.array(self.img_label[index]))
    def __len__(self):
        return len(self.img_path)
        
import albumentations as A
# 训练集
train_loader = torch.utils.data.DataLoader(
    XunFeiDataset(train_path[:-1000], train_label[:-1000],
            A.Compose([
            # A.RandomRotate90(),
            A.Resize(256, 256),
            A.RandomCrop(224, 224),
            # A.HorizontalFlip(p=0.5),
            # A.RandomBrightnessContrast(p=0.5),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
    ), batch_size=30, shuffle=True, num_workers=0, pin_memory=False
)

# 验证集
val_loader = torch.utils.data.DataLoader(
    XunFeiDataset(train_path[-1000:], train_label[-1000:],
            A.Compose([
            A.Resize(256, 256),
            A.RandomCrop(224, 224),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
    ), batch_size=50, shuffle=False, num_workers=0, pin_memory=False
)

# 测试集
test_loader = torch.utils.data.DataLoader(
    XunFeiDataset(test_path, [0] * len(test_path),
            A.Compose([
            A.Resize(256, 256),
            A.RandomCrop(224, 224),
            # A.HorizontalFlip(p=0.5),
            # A.RandomBrightnessContrast(p=0.5),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
    ), batch_size=50, shuffle=False, num_workers=0, pin_memory=False
)

In [20]:
class XunFeiNet(nn.Module):
    def __init__(self):
        super(XunFeiNet, self).__init__()
        model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(2048, 25)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out
        
model = XunFeiNet()
model = model.to(device)
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.AdamW(model.parameters(), 0.001)

In [21]:
# 模型训练
def train(train_loader, model, criterion, optimizer):
    start = time.time()
    start_batch = [start, 0]
    model.train()
    train_loss = 0.0
    preds = torch.tensor([])
    target_all = torch.tensor([])
    for i, (input, target) in enumerate(train_loader):
        input = input.to(device)
        target = target.to(device)
        output = model(input)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            start_batch[((i + 1) // 100) % 2] = time.time()
            print(
                "Train loss",
                loss.item(),
                "t={}s".format(
                    start_batch[((i + 1) // 100) % 2]
                    - start_batch[(1 + (i + 1) // 100) % 2]
                ),
            )

        preds = torch.cat((preds, output.cpu().argmax(1)))
        target_all = torch.cat((target_all, target.cpu()))

        train_loss += loss.item()

    val_acc = torchmetrics.functional.classification.multiclass_f1_score(
        preds, target_all, num_classes=25, average="macro"
    )
    print("t={}s".format(time.time() - start))
    print("F1 score", val_acc)
    return train_loss / len(train_loader)


# 模型验证
def validate(val_loader, model):
    model.eval()
    val_acc = 0.0
    preds = torch.tensor([])
    target_all = torch.tensor([])
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            input = input.to(device)
            target = target.to(device)
            output = model(input).cpu()
            # val_acc += (output.argmax(1) == target).sum().item()
            preds = torch.cat((preds, output.argmax(1)))
            target_all = torch.cat((target_all, target.cpu()))

        val_acc = torchmetrics.functional.classification.multiclass_f1_score(
            preds, target_all, num_classes=25, average="macro"
        )

    # return val_acc / len(val_loader.dataset)
    return val_acc


# 模型预测
def predict(test_loader, model):
    model.eval()

    test_pred = []
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            input = input.to(device)
            target = target.to(device)
            output = model(input)
            test_pred.append(output.data.cpu().numpy())

    return np.vstack(test_pred)

In [22]:
epochs = 0
last_acc = 0
val_acc = 0.001
while last_acc < val_acc:
    last_acc = val_acc
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc = validate(val_loader, model)
    # train_acc = validate(train_loader, model)
    print(
        "epoch {} :".format(epochs + 1), "train_loss:", train_loss, "val_f1:", val_acc
    )
    epochs += 1
    if val_acc>last_acc:
        torch.save(model.state_dict(),"./model/baseline.pth")

Train loss 2.2976112365722656 t=16.140373468399048s
Train loss 2.1149075031280518 t=16.00533151626587s
Train loss 1.7303135395050049 t=16.142765045166016s
Train loss 1.8303672075271606 t=16.645331144332886s
Train loss 1.4413533210754395 t=16.39311146736145s
Train loss 1.281042456626892 t=17.26898956298828s
Train loss 1.3583972454071045 t=17.249473094940186s
t=121.25433492660522s
F1 score tensor(0.4267)
epoch 1 : train_loss: 1.744115650490762 val_f1: tensor(0.5191)
Train loss 0.8689115643501282 t=16.13426971435547s
Train loss 0.9837633371353149 t=15.964512825012207s
Train loss 0.9596181511878967 t=16.01486325263977s
Train loss 1.0082253217697144 t=15.850048780441284s
Train loss 0.839529275894165 t=15.899014949798584s
Train loss 0.3169907331466675 t=15.859082460403442s
Train loss 0.5702284574508667 t=15.73872709274292s
t=116.4113380908966s
F1 score tensor(0.7129)
epoch 1 : train_loss: 0.8117695489921257 val_f1: tensor(0.7750)
Train loss 0.3554437756538391 t=15.643813133239746s
Train loss

In [23]:
# 对测试集多次预测
pred = None
model.load_state_dict(torch.load("./model/baseline.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
submit = pd.DataFrame(
    {
        'name': [x.split('/')[-1] for x in test_path],
        'label': pred.argmax(1)
})

# 生成提交结果
submit = submit.sort_values(by='name')
submit.to_csv('submit.csv', index=None)

1
2
3


In [ ]:
model.train()
train_loss = 0.0
model = model.to(device)
for i, (input, target) in enumerate(train_loader):
    input = input.to(device)
    target = target.to(device)
    output = model(input)
    loss = criterion(output, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model.fc

In [ ]:
val_acc = validate(val_loader, model)
val_acc